In [1]:
import json
import os
import glob

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series,DataFrame

#sklearn.cluster.
import pickle
import logging
from time import time
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from scipy import stats
from sklearn.cluster import KMeans
from soyclustering import SphericalKMeans


In [2]:
train=pd.read_json('res/test.json',typ='frame')
train

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000
1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000
2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000
3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000
4,[],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000
...,...,...,...,...,...,...
10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000
10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000
10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000
10738,[여름],2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",4,2015-06-06 09:52:01.000


In [9]:
val_songs=train[['id','tags','songs']]
songs=[]
tags=[]
for i in range(10740):
    songs.append(len(val_songs['songs'][i]))
songs=pd.DataFrame({'len_song':songs})
songs
for j in range(10740):
    tags.append(len(val_songs['tags'][j]))
tags=pd.DataFrame({'len_tag':tags})
tags

,len_tag
0,0
1,2
2,1
3,1
4,0
...,...
10735,1
10736,5
10737,1
10738,1


In [12]:
m=pd.concat([val_songs, songs,tags], axis=1)
m

,id,tags,songs,len_song,len_tag
0,70107,[],"[398985, 449403, 411543, 528044, 143048, 98020...",28,0
1,7461,"[나만의Best3, 인디아티스트들의추천음악]","[196298, 269984, 267805, 175867, 529244, 63825...",8,2
2,90348,[드라이브],"[273433, 331003, 68432, 411659, 117793, 616860...",40,1
3,58617,[분위기],"[702227, 48152, 440008, 358488, 701041, 540721...",23,1
4,102395,[],"[630683, 481582, 528550, 285114, 506667, 17922...",15,0
...,...,...,...,...,...
10735,137930,[추억],"[323755, 397594, 445908, 570242, 221853, 20018...",34,1
10736,936,"[띵곡의, 우울, 분위기, 드라이브, 산책]","[105140, 582252, 199262, 422915, 547967, 48791...",37,5
10737,110589,[기분전환],"[21976, 207746, 40025, 31635, 567462, 641799, ...",15,1
10738,2605,[여름],"[234554, 265033, 507260, 83092, 366757, 497097...",19,1


In [32]:
data='len_song < 1'
data1='len_tag < 1'
data2='len_tag > 0'
data3='len_song > 0'
#태그만 없는 df
none_song = m.query(data) 
none_song_df = none_song.query(data2)
none_song_df=none_song_df[['id','tags','songs']]
#노래만 없는 df
none_tag = m.query(data1)
none_tag_df = none_tag.query(data3)
none_tag_df=none_tag_df[['id','tags','songs']]
#노래랑 tag 둘다 없는 df
none_song = m.query(data)
none_song_tag_df = none_song.query(data1)
none_song_tag_df=none_song_tag_df[['id','tags','songs']]

In [33]:
#song만 없는
none_song_df

,id,tags,songs
5,88862,[스트레스],[]
7,133975,[사랑],[]
11,135323,"[재즈, 기분전환, 뉴에이지]",[]
21,113806,[새벽],[]
30,126210,"[Pop, 산책, 인디]",[]
...,...,...,...
10689,113125,"[봄캐롤, 긱스, 버스커버스커, 봄]",[]
10707,27982,[추억],[]
10709,123364,[사랑],[]
10712,61817,[기분전환],[]


In [34]:
#태그만 없는
none_tag_df

,id,tags,songs
0,70107,[],"[398985, 449403, 411543, 528044, 143048, 98020..."
4,102395,[],"[630683, 481582, 528550, 285114, 506667, 17922..."
6,42586,[],"[538867, 513110, 337317, 225168, 211527, 42507..."
8,13244,[],"[32042, 98472, 475222, 56796, 662086, 666787, ..."
10,55399,[],"[148079, 651833, 388619, 264320, 347391, 49002..."
...,...,...,...
10730,120081,[],"[249220, 88841, 359359, 224607, 298719, 262021..."
10731,16489,[],"[260730, 31312, 250159, 374880, 243178, 19411,..."
10732,46470,[],"[67655, 65672, 210647, 608386, 413837, 566468,..."
10734,48788,[],"[145292, 402779, 452316, 324334, 217189, 42301..."


In [35]:
none_song_tag_df

,id,tags,songs
17,43130,[],[]
38,17669,[],[]
40,20179,[],[]
49,133553,[],[]
70,131124,[],[]
...,...,...,...
10667,114894,[],[]
10678,90786,[],[]
10686,6756,[],[]
10701,80375,[],[]


In [40]:
none_tag_df.to_json('none_tag_df.json', orient='table')
none_song_df.to_json('none_song_df.json', orient='table')
none_song_tag_df.to_json('none_song_tag_df.json', orient='table')

In [42]:
none = pd.read_json('none_song_tag_df.json', orient='table')

In [43]:
none

,id,tags,songs
17,43130,[],[]
38,17669,[],[]
40,20179,[],[]
49,133553,[],[]
70,131124,[],[]
...,...,...,...
10667,114894,[],[]
10678,90786,[],[]
10686,6756,[],[]
10701,80375,[],[]
